# word2vec implementation

## Original papers
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)
* [word2vec Explained: deriving Mikolov et al.'s negative-sampling word-embedding method](https://arxiv.org/abs/1402.3722)



## Overview

The original paper proposed using:
1. Matmul to extract word vectors from ```Win```.
2. Softmax to calculate scores with all the word vectors from ```Wout```.

<img src="image/word2vec_cbow_mechanism.png" align="left"/>

## Negative Sampling

### Motivation

The original implementation was computationally expensive.

1. Matmal to extract word vectors from ```Win```.
2. Softmax can happen vocabulary size times with ```Wout```.

<img src="image/word2vec_negative_sampling_motivation.png" align="left"/>


### Solution

1. Use index to extract word vectors from Win.
2. Instead of calculating softmax with all the word vectors from ```Wout```, sample small number (SL) of negative/false word vectors from ```Wout``` and calculate logistic log loss with each sample. 

<img src="image/wors2vec_neg_sample_backprop.png" align="left"/>

## Using only one Word vector space W

There is no reasoning nor proof why two word vector space are required. In the end, we only use one word vector space, which appears to be ```Win```. 

However, if we use one vector space ```W``` for ```event```, ```context``` and ```negative samples```,then an event vector ```event=W[i]``` in a sentence can be used as a negative sample in another setence. Then the weight ```W[i]``` is updated for both positive and negative labels in the same gradient descent on ```W```. The actual [experiment of using only one vector space](./layer/embedding_single_vector_space.py) ```W``` did not work well.

* [Why do we need 2 matrices for word2vec or GloVe](https://datascience.stackexchange.com/a/94422/68313)


<img src="image/word2vec_why_not_one_W.png" align="left" width=800/>

---
# Setups

In [1]:
import cProfile
import sys
import os
import time
import re
from itertools import islice
from typing import Dict, List
import numpy as np
import tensorflow as tf

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=400) 

## Setup for Google Colab environment

Colab gets disconnected within approx 20 min. Hence not suitable for training (or need to upgrade to the pro version).

### Clone github to Google Drive

In [2]:
try:
    import google.colab
    IN_GOOGLE_COLAB = True
except:
    IN_GOOGLE_COLAB = False
    
if IN_GOOGLE_COLAB:
    !pip install line_profiler
    !google.colab.drive.mount('/content/gdrive')
    !rm -rf /content/drive/MyDrive/github
    !mkdir -p /content/drive/MyDrive/github
    !git clone https://github.com/oonisim/python-programs.git /content/drive/MyDrive/github


### Clone github to local directory

In [3]:
try:
    import google.colab
    IN_GOOGLE_COLAB = True
except:
    IN_GOOGLE_COLAB = False
    
if IN_GOOGLE_COLAB:
    !pip install line_profiler
    !google.colab.drive.mount('/content/gdrive')
    !rm -rf /content/github
    !mkdir -p /content/github
    !git clone https://github.com/oonisim/python-programs.git /content/github
        
    import sys
    sys.path.append('/content/github/nlp/src')

# Jupyter notebook setups

Auto reolaod causes an error in Jupyter notebooks. Restart the Jupyter kernel for the error:
```TypeError: super(type, obj): obj must be an instance or subtype of type```
See
- https://stackoverflow.com/a/52927102/4281353
- http://thomas-cokelaer.info/blog/2011/09/382/

> The problem resides in the mechanism of reloading modules.
> Reloading a module often changes the internal object in memory which
> makes the isinstance test of super return False.

In [4]:
%load_ext line_profiler
%load_ext autoreload

## Utilites

In [5]:
%autoreload 2

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

import function.fileio as fileio
import function.text as text

---
# Data Types


In [6]:
from common.constant import (
    TYPE_INT,
    TYPE_FLOAT,
    TYPE_LABEL,
    TYPE_TENSOR,
)

# Constants

In [7]:
USE_PTB = False
USE_TEXT8 = True

CORPUS_FILE = "text8_512" if USE_TEXT8 else "ptb_train"
CORPUS_URL = "https://data.deepai.org/text8.zip" \
    if USE_TEXT8 else f'https://raw.githubusercontent.com/tomsercu/lstm/master/data/ptb.train.txt' \

TARGET_SIZE = TYPE_INT(1)       # Size of the target event (word)
CONTEXT_SIZE = TYPE_INT(10) # Size of the context in which the target event occurs.
WINDOW_SIZE = TARGET_SIZE + CONTEXT_SIZE
SAMPLE_SIZE = TYPE_INT(5)       # Size of the negative samples

VECTOR_SIZE = TYPE_INT(100)     # Number of features in the event vector.
WEIGHT_SCHEME = "normal"
WEIGHT_PARAMS = {
    "std": 0.01
}
LR = TYPE_FLOAT(100.0)

NUM_SENTENCES = 1

STATE_FILE = "../models/word2vec_%s_E%s_C%s_W%s_%s_%s_V%s_LR%s_S%s_N%s.pkl" % (
    CORPUS_FILE,
    TARGET_SIZE,
    CONTEXT_SIZE,
    WEIGHT_SCHEME,
    "std",
    WEIGHT_PARAMS["std"],
    VECTOR_SIZE,
    LR,
    SAMPLE_SIZE,
    NUM_SENTENCES,
)

MAX_ITERATIONS = 100000

---

# Data
## Corpus

In [9]:
path_to_corpus = f"~/.keras/datasets/{CORPUS_FILE}"
if fileio.Function.is_file(path_to_corpus):
    pass
else:
    # text8, run "cat text8 | xargs -n 512 > text8_512" after download
    path_to_corpus = tf.keras.utils.get_file(
        fname=CORPUS_FILE,
        origin=CORPUS_URL,
        extract=True
    )
corpus = fileio.Function.read_file(path_to_corpus)
print(path_to_corpus)

/home/oonisim/.keras/datasets/text8_512


In [ ]:
examples = corpus.split('\n')[:1]
for line in examples:
    print(line)

---
# Event (word) indexing
Index the events that have occurred in the event sequence.

In [10]:
%autoreload 2
from layer.preprocessing import (
    EventIndexing
)

/home/oonisim/conda/envs/python_programs/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [11]:
word_indexing = EventIndexing(
    name="word_indexing",
    corpus=corpus,
    min_sequence_length=WINDOW_SIZE
)
del corpus

## EventIndexing  for the corpus

Adapt to the ```corpus``` and provides:
* event_to_index dictionary
* vocaburary of the corpus
* word occurrence probabilites

In [12]:
words = word_indexing.list_events(range(10))
print(f"EventIndexing.vocabulary[10]:\n{words}\n")

indices = word_indexing.list_indices(words)
print(f"EventIndexing.event_to_index[10]:")
for item in zip(words, indices):
    print(item)

probabilities = word_indexing.list_probabilities(words)
print(f"\nEventIndexing.probabilities[10]:")
for word, p in zip(words, probabilities):
    print(f"{word:20s} : {p:.5e}")

EventIndexing.vocabulary[10]:
['<nil>' '<unk>' 'anarchism' 'originated' 'as' 'a' 'term' 'of' 'abuse' 'first']

EventIndexing.event_to_index[10]:
('<nil>', 0)
('<unk>', 1)
('anarchism', 2)
('originated', 3)
('as', 4)
('a', 5)
('term', 6)
('of', 7)
('abuse', 8)
('first', 9)

EventIndexing.probabilities[10]:
<nil>                : 0.00000e+00
<unk>                : 0.00000e+00
anarchism            : 2.91638e-05
originated           : 4.69688e-05
as                   : 2.77802e-03
a                    : 5.47707e-03
term                 : 3.14500e-04
of                   : 8.58865e-03
abuse                : 4.64134e-05
first                : 8.88014e-04


## Sampling using the probability

Sample events according to their probabilities.

In [13]:
sample = word_indexing.sample(size=5)
print(sample)

['lombok', 'replacement', 'mishnah', 'international', 'on']


## Negative Sampling
Sample events not including those events already sampled.

In [14]:
negative_indices = word_indexing.negative_sample_indices(
    size=5, excludes=word_indexing.list_indices(sample)
)
print(f"negative_indices={negative_indices} \nevents={word_indexing.list_events(negative_indices)}")

negative_indices=[4257, 60581, 2567, 7850, 461] 
events=['plot' 'participatory' 'iq' 'particle' 'years']


## Sentence to Sequence

In [15]:
# sentences = "\n".join(corpus.split('\n')[5:6])
sentences = """
the asbestos fiber <unk> is unusually <unk> once it enters the <unk> 
"""
sequences = word_indexing.function(sentences)
for pair in zip(sentences.strip().split(" "), sequences[0]):
    print(f"{pair[0]:15} : {pair[1]:5}")

Sentence is empty. Skipping...
Sentence is empty. Skipping...


the             :    17
asbestos        : 61541
fiber           : 15042
<unk>           :     1
is              :    26
unusually       :  2241
<unk>           :     1
once            :  2296
it              :    41
enters          :  2635
the             :    17
<unk>           :     1


---
# EventContext

EventContext layer generates ```(event, context)``` pairs from sequences.

In [16]:
%autoreload 2
from layer.preprocessing import (
    EventContext
)

In [17]:
event_context = EventContext(
    name="ev",
    window_size=WINDOW_SIZE,
    event_size=TARGET_SIZE
)

## Context of an event (word) in a sentence

In the sentence ```"a form of asbestos once used to make kent cigarette filters"```, one of the context windows ```a form of asbestos once``` of size 5 and event size 1 has.
* ```of``` as a target word.
* ```(a, form) and (asbestos, once)``` as its context.

### Sequence of the word indices for the sentence

In [18]:
sentences = """
a form of asbestos once used to make kent cigarette filters

N years old and former chairman of consolidated gold 
"""

sequence = word_indexing.function(sentences)
sequence

Sentence is empty. Skipping...
Sentence is empty. Skipping...
Sentence is empty. Skipping...


array([[    5,   410,     7, 61541,  2296,    10,    31,  1239, 25481,  8773, 17383],
       [  783,   461,  1132,    21,  2601,  7508,     7, 16791,  4204,     0,     0]])

## (event, context) pairs

For each word (event) in the setence ```(of, asbestos, ... , kent)``` excludnig the ends of the sentence, create ```(target, context)``` as:

```
[
  [of, a, form, asbestos, once],              # target is 'of', context is (a, form, asbestos, once)
  ['asbestos', 'form', 'of', 'once', 'used'],
  ['once', 'of', 'asbestos', 'used', 'to'],
  ...
]
```

### Format of the (event, context) pairs

* **E** is the target event indices
* **C** is the context indices

<img src="image/event_context_format.png" align="left"/>

In [19]:
event_context_pairs = event_context.function(sequence)
print(
    f"Event context pairs. Shape %s, Target event size %s, Window size %s." % 
    (event_context_pairs.shape, event_context.event_size, event_context.window_size)
)
event_context_pairs[:10]

Event context pairs. Shape (2, 11), Target event size 1, Window size 11.


array([[   10,     5,   410,     7, 61541,  2296,    31,  1239, 25481,  8773, 17383],
       [ 7508,   783,   461,  1132,    21,  2601,     7, 16791,  4204,     0,     0]], dtype=int32)

### (event, context) pairs in textual words

In [20]:
word_indexing.sequence_to_sentence(event_context_pairs[:10])

[['used',
  'a',
  'form',
  'of',
  'asbestos',
  'once',
  'to',
  'make',
  'kent',
  'cigarette',
  'filters'],
 ['chairman',
  'n',
  'years',
  'old',
  'and',
  'former',
  'of',
  'consolidated',
  'gold',
  '<nil>',
  '<nil>']]

---
# Word Embedding

Embedding is to train the model to group similar events in a close proximity in the event vector space. If two events e.g. 'pencil' and 'pen' are similar concepts, then their event vectors resides in a close distance in the event space. 

* [Thought Vectors](https://wiki.pathmind.com/thought-vectors)

## Training process

1. Calculate ```Bc```, the BoW (Bag of Words) from context event vectors.
2. Calculate ```Be```,  the BoW (Bag of Words) from target event vectors.
3. The dot product ```Ye = dot(Bc, Be)``` is given the label 1 to get them closer.
4. For each negative sample ```Ws(s)```, the dot product with ```Ys = dot(Be, Ws(s)``` is given the label 0 to get them apart. 
5. ```np.c_[Ye, Ys]``` is fowarded to the logistic log loss layer.

<img src="image/word2vec_backprop_Be.png" align="left"/>


In [21]:
%autoreload 2
from layer import (
    Embedding
)
from optimizer import (
    SGD
)

In [22]:
embedding: Embedding = Embedding(
    name="embedding",
    num_nodes=WINDOW_SIZE,
    target_size=TARGET_SIZE,
    context_size=CONTEXT_SIZE,
    negative_sample_size=SAMPLE_SIZE,
    event_vector_size=VECTOR_SIZE,
    optimizer=SGD(lr=LR),
    dictionary=word_indexing,
    weight_initialization_scheme=WEIGHT_SCHEME,
    weight_initialization_parameters=WEIGHT_PARAMS
)

### Scores ```np.c_[Ye, Ys]``` from the Embedding

The 0th column is the scores for ```dot(Bc, Be``` for positive labels. The rest are the scores for ```dot(Bc, Ws)``` for negative labels.

In [23]:
scores = embedding.function(event_context_pairs)
print(f"Scores:\n{scores[:5]}\n")
print(f"Scores for dot(Bc, Be): \n{scores[:5, :1]}")

Scores:
[[ 3.8861856e-04 -8.2609113e-05 -1.6636041e-06 -2.7692990e-04 -2.5679212e-04  1.7823913e-04]
 [-2.5097700e-04  1.4876407e-04  4.9260627e-05  7.2004477e-05 -5.7879178e-04  6.1351864e-05]]

Scores for dot(Bc, Be): 
[[ 0.00038862]
 [-0.00025098]]


---
# Logistic Log Loss

Train the model to get:
1. BoW of the context event vectors close to the target event vector. Label 1
2. BoW of the context event vectors away from each of the negative sample event vectors Label 0.

This is a binary logistic classification, hence use Logistic Log Loss as the network objective function.

In [24]:
from common.function import (
    sigmoid_cross_entropy_log_loss,
    sigmoid
)
from layer.objective import (
    CrossEntropyLogLoss
)

In [25]:
loss = CrossEntropyLogLoss(
    name="loss",
    num_nodes=1,  # Logistic log loss
    log_loss_function=sigmoid_cross_entropy_log_loss
)

---
# Adapter

The logistic log loss layer expects the input of shape ```(N,M=1)```, however Embedding outputs ```(N,(1+SL)``` where ```SL``` is SAMPLE_SIZE. The ```Adapter``` layer bridges between Embedding and the Loss layers.


In [26]:
from layer.adapter import (
    Adapter
)

In [27]:
adapter_function = embedding.adapt_function_to_logistic_log_loss(loss=loss)
adapter_gradient = embedding.adapt_gradient_to_logistic_log_loss()

adapter: Adapter = Adapter(
    name="adapter",
    num_nodes=1,    # Number of output M=1 
    function=adapter_function,
    gradient=adapter_gradient
)

---
# Word2vec Network

## Construct a sequential network

$ \text {Sentences} \rightarrow EventIndexing \rightarrow EventContext \rightarrow  Embedding \rightarrow Adapter \rightarrow LogisticLogLoss$

In [28]:
from network.sequential import (
    SequentialNetwork
)

In [29]:
network = SequentialNetwork(
    name="word2vec",
    num_nodes=1,
    inference_layers=[
        word_indexing,
        event_context,
        embedding,
        adapter
    ],
    objective_layers=[
        loss
    ]
)

## Run training

### Load the saved model file if exists

In [31]:
if fileio.Function.is_file(STATE_FILE):
    print("Loading model...\nSTATE_FILE: %s" % STATE_FILE)
    state = embedding.load(STATE_FILE)

    fmt="""Model loaded.
    event_size %s
    context_size: %s
    event_vector_size: %s
    """
    print(fmt % (
        state["target_size"], 
        state["context_size"], 
        state["event_vector_size"]
    ))
else:
    print("State file does not exist. Saving the initial model to %s." % STATE_FILE)
    embedding.save(STATE_FILE)

State file does not exist. Saving the initial model to ../models/word2vec_text8_512_E1_C10_Wnormal_std_0.01_V100_LR100.0_S5_N1.pkl.


### Continue training

In [ ]:
def sentences_generator(path_to_file, num_sentences):
    stream = fileio.Function.file_line_stream(path_to_file)
    try:
        while True:
            _lines = fileio.Function.take(num_sentences, stream)
            yield np.array(_lines)
    finally:
        stream.close()

# Sentences for the trainig
source = sentences_generator(
    path_to_file=path_to_corpus, num_sentences=NUM_SENTENCES
)

# Restore the state if exists.
state = embedding.load(STATE_FILE)

# Continue training
profiler = cProfile.Profile()
profiler.enable()

total_sentences = 0
epochs = 0

for i in range(MAX_ITERATIONS):
    try:
        sentences = next(source)
        total_sentences += len(sentences)

        start = time.time()
        network.train(X=sentences, T=np.array([0]))

        if i % 100 == 0:
            print(
                f"Batch {i:05d} of {NUM_SENTENCES} sentences: "
                f"Average Loss: {np.mean(network.history):10f} "
                f"Duration {time.time() - start:3f}"
            )
        if i % 100 == 0:
            # embedding.save(STATE_FILE)
            pass
        
    except fileio.Function.GenearatorHasNoMore as e:
        # Next epoch
        print(f"epoch {epochs} batches {i:05d} done")
        print(f"Average loss: {np.mean(network.history):15f}")
        epochs += 1
        source.close()
        source = sentences_generator(
            path_to_file=path_to_corpus, num_sentences=NUM_SENTENCES
        )

    except Exception as e:
        print("Unexpected error:", sys.exc_info()[0])
        source.close()
        raise e

embedding.save(STATE_FILE)

profiler.disable()
profiler.print_stats(sort="cumtime")

Batch 00000 of 1 sentences: Average Loss:   0.693139 Duration 2.243656
Batch 00100 of 1 sentences: Average Loss:   0.631834 Duration 1.855242
Batch 00200 of 1 sentences: Average Loss:   0.548606 Duration 1.848966
Batch 00300 of 1 sentences: Average Loss:   0.504841 Duration 1.867725
Batch 00400 of 1 sentences: Average Loss:   0.476411 Duration 1.955096
Batch 00500 of 1 sentences: Average Loss:   0.456340 Duration 2.340900
Batch 00600 of 1 sentences: Average Loss:   0.440056 Duration 1.885046
Batch 00700 of 1 sentences: Average Loss:   0.427696 Duration 2.258552
Batch 00800 of 1 sentences: Average Loss:   0.417222 Duration 2.127491
Batch 00900 of 1 sentences: Average Loss:   0.408238 Duration 1.890859


---
# Evaluate the vector space

Verify if the trained model, or the vector space W, has encoded the words in a way that **similar** words are close in the vector space.

* [How to measure the similarity among vectors](https://math.stackexchange.com/questions/4132458)

In [ ]:
n = 10
context = "cash".split()
word_indices = np.array(word_indexing.list_indices(context), dtype=TYPE_INT)

print(f"Words {context}")
print(f"Word indices {word_indices}")
print(f"prediction for {context}:\n{word_indexing.list_events([embedding.predict(word_indices, n)])}")

In [ ]:
np.zeros(shape=(141, 11))

In [ ]:
---
## Compare with [gensim word2vec](https://radimrehurek.com/gensim/models/word2vec.html)

In [ ]:
from gensim.models import (
    Word2Vec
)
from gensim.models.word2vec import (
    LineSentence    
)

In [ ]:
sentences = LineSentence(source=path_to_corpus)
w2v = Word2Vec(
    sentences=sentences, 
    sg=0,
    window=5, 
    negative=5,
    vector_size=100, 
    min_count=1, 
    workers=4
)
del sentences

In [ ]:
w2v.wv.most_similar(context, topn=n)